# Run the code to get player data in list

In [ ]:
import time
import pandas as pd
import unidecode

# create the alphabet in a string in order to traverse through letter 
alphabet = 'abcdefghijklmnopqrstuvwxyz'
base_url = 'https://www.basketball-reference.com/players'
temp = []

for letter in alphabet:
    time.sleep(2)
    df = pd.read_html(f"{base_url}/{letter}/")[0]
    df['Letter'] = [letter]*len(df)
    temp.append(df)
    print(letter)
    
data = pd.concat(temp)
data = data.reset_index(drop = True)

data.to_csv('all_players.csv', index = False) 

# Get data from csv file instead

In [ ]:
import time
import pandas as pd
import unidecode

data = pd.read_csv('all_players.csv')
data

# Get stars and middleNames

In [ ]:
starPlayers = [player for player in data.Player if player[-1] == '*']
players = [unidecode.unidecode(player) if player not in starPlayers else unidecode.unidecode(player[0:-1]) for player in data.Player]
data.Player = players
middleNames = [name for name in data.Player if (len(name.split(' '))>2)]

# Set up loop to get player dataframes

In [ ]:
base_url = 'https://www.basketball-reference.com/players'
CornerCases={
    "Luc Mbah a Moute":  "mbahalu",
    "Metta World Peace": "artesro",
    "Logan Vander Velden": "vandelo",
    "Nene": "hilarne",
    "Jeff Ayres": "pendeje",
    "Mark Baker": "bakerla",
    "J.J. Barea": "bareajo",
    "George Bon Salle": "bonsage", ## Could check for bon with the vans and vons
    "Clint Capela": "capelca",
    "Dick Clark": "clarkri",
    "Gigi Datome": "datomlu",
    "Vinny Del Negro": "delnevi",
    "Nando De Colo": "decolna",  ## and dels
    "Rich Eichhorst": "eichhdi",
    "Wayne Engelstad": "englewa",
    "Bill Franklin": "frankwi",
    "Enes Freedom": "kanteen",
    "Smokey Gaines": "gaineda",
    "Gene Gallette": "gillege",
    "Bubbles Hawkins": "hawkiro",
    "Walt Hazzard": "abdulma",
    "Paul Hogue": "hoguedu",
    "Alfredrick Hughes": "hugheri",
    "Howie Jolliff": "jolifho",
    "Maxi Kleber": "klebima",
    "Didi Louzada": "louzama",
    "Sheldon Mac": "mcclesh",
    "D.J. Mbenga": "mbengdj",
    "Frank Ntilikina": "ntilila",
    "Cedi Osman": "osmande",
    "Sasha Pavlovic": "pavloal",
    "Jim Phelan": "phelaja",
    "Don Putman": "putnado",
    "Bob Rensberger": "rensbro",
    "Mouhamed Sene": "senesa",
    "Edy Tavares": "tavarwa",
    "Gene Tormohlen": "tormobu",
    "Marcus Vinicius": "vincima",
    "Henry Walker": "walkebi",
    "Mo Williams": "willima",
    "Jimmy Foster": "fosteja",
    "KJ Martin": "martike",
    "Taylor Hendricks": "hendrita",
    "Sasha Vezenkov": "vezenal",
    "Tristan Da Silva": "dasiltr",
}
extensions = []


for i in range(len(data)):
    
    player = data.iloc[i].Player
    FullName = unidecode.unidecode(player)
    
    if FullName in CornerCases.keys():
        extension = CornerCases[FullName]
        data.at[i,'Letter'] = extension[0]
    else:
        
        FullName = FullName.replace("'","")
        FullName = FullName.replace(".","")
        FullName = FullName.replace("-","")
        letter = data.iloc[i].Letter
        name_array = FullName.split(' ')
        
        if len(name_array)>2:
            first2 = name_array[0][0:2].lower()
            

            if name_array[1].lower() == "von" or (name_array[1].lower()=="van"): #check for the dutch
                first5 = name_array[1][0:3].lower() + name_array[2][0:2].lower()
            elif name_array[2].lower() == 'jr': #Check for juniors
                first5 = name_array[1][0:5].lower()
            else:
                for j in range(len(name_array)-1, 0, -1):
                    if name_array[j][0].lower() == letter:
                        first5= name_array[j][0:5].lower()
                        break

        else:
            first5 = name_array[1][0:5].lower()
            first2 = name_array[0][0:2].lower()

        extension = first5 + first2

    extensions.append(extension)

data['Extension'] = extensions    

In [ ]:
print("FullName: ", FullName)
print("extension: ", extension)
print("letter: ", letter)
print("Possible url: ", f'{base_url}/{letter}/{extension}01.html')

# Check duplicate extensions

In [ ]:
########Put logic to get columns for number#############
data = data.sort_values(['Extension','From']).reset_index(drop = True)

CornerCaseNumbers = {
    "hairspj01": 2,
    "munfoxa01": 2,
    "brownch03": 5,
    "brownke02": 3,
    "burtode01": 2,
    "davisjo03": 6,
    "fostemi01": 2,
    "greenja03": 5,
    "howarma01": 2,
    "jacksjo01": 2,
    "johnsda05": 8,
    "johnsja03": 5,
    "johnske05": 7,
    "jonesca02": 3,
    "jonesja03": 4,
    "jonesma04": 5,
    "smithja03": 4,
    "smithto01": 2,
    "tilliki01": 2,
    "washity01": 2,
    "willial04": 6,
    "willibr02": 3,
    "willija05": 6,
    "willima06": 7,
    "williro03": 4,
    "willido01": 2,
    "lewisma02": 5,
    "millejo01": 2,
    "mooreta01": 2,
    "willima08": 11,
    "jamesbr01": 2,
    "smithty01": 2,
    "willico03": 4,
}

resetNum = 1
prevNum = 1
numbers = [1]
currentNum = 1

for i in range(1,len(data)):
    if data.iloc[i].Extension == data.iloc[i-1].Extension:
        currentNum = prevNum + 1
    else: 
        currentNum = resetNum
    
    key = f"{data.iloc[i].Extension}{str(currentNum).zfill(2)}"
    if key in CornerCaseNumbers.keys(): 
        currentNum = CornerCaseNumbers[key]
    
    prevNum = currentNum
    numbers.append(currentNum)
    
numbers = [str(number).zfill(2) for number in numbers]
data['Number'] = numbers

# Generate the correct urls (and verify no duplicates)

In [ ]:
urls = []

for i in range(len(data)):
    row = data.iloc[i]
    url = f"{base_url}/{row.Letter}/{row.Extension}{row.Number}.html"
    urls.append(url)
    
data['url'] = urls
assert(len(data['url'].unique()) == len(data))

# Read htmls and write csvs

In [ ]:
import time
import os

len_data = len(data)
bad_rows = []

raw_dir = 'raw_player_dfs/'
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

for i in range(len_data):
# for i in range(len(bad_df)):
    
    row = data.iloc[i]
#     row = bad_df.iloc[i]
    path = f'raw_player_dfs/{row.Letter}'
    
    if not os.path.exists(path):
        os.makedirs(path)
    file = f"{path}/{row.Extension}{row.Number}.csv"

    if row.To != 2025: ####PUT CONDITION FOR CURRENT PLAYERS (also for skipping) if row.Player != "Tristan Da Silva"
        continue
    else:
        try:
            time.sleep(2)
            print(f"{i+1}/{len_data}", row.Player)
            j = 0
            bbref_html = pd.read_html(row.url)
            player_df = bbref_html[j]
            while player_df.columns[0] != 'Season':
                j+=1
                player_df = bbref_html[j]
            player_df.to_csv(file, index = False)
        except:
            print(f'Did not work at index {i}:', row.Player)
            bad_rows.append(row)
    
            
try:
    bad_df = pd.concat(bad_rows,axis =1).T
except:
    print("All players generated! Yay!")

# Clean up dataframe

In [ ]:
import os

alphabet = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = alphabet.upper()

clean_dir = 'player_dfs/'
if not os.path.exists(clean_dir):
    os.makedirs(clean_dir)
    
len_data = len(data)
bad_rows = []
    
for index,row in data.iterrows():
    
    print(f'{index+1}/{len_data}',row.Player)
    
    try:
        path = f'{clean_dir}{row.Letter}'
        if not os.path.exists(path):
            os.makedirs(path)
    #####################################READ IN RAW PLAYER DF####################################
        file = f'{row.Letter}/{row.Extension}{row.Number}.csv'
        raw_player_df = pd.read_csv(f'raw_player_dfs/{file}')
        player_df = raw_player_df.dropna(subset = ['Pos'])

    ####################################CHECK FOR DNP SEASONS################################
        if player_df.G.dtype == 'object':
            DNP = [False if (row[1].G[0] in alphabet) or (row[1].G[0] in ALPHABET) else True for row in player_df.iterrows()]
            player_df = player_df[DNP]

    ########################DROP DUPLICATE SEASON TO GET TOT#####################################
        player_df = player_df.drop_duplicates(subset = ['Season'], keep = 'first')
        player_df = player_df.reset_index(drop = True)

    ############################WRITE THE CSV FILE#######################################
        file = f'{clean_dir}{file}'
        player_df = player_df.set_index('Season')
        player_df['Player'] = [row.Player]*len(player_df)
        player_df['Pos'] = [row.Pos]*len(player_df)
        player_df['Ht'] = [row.Ht]*len(player_df)
        player_df.to_csv(file)
    
    except:
        print(f'Did not work at index {index}:', row.Player)
        bad_rows.append(row)

try:
    bad_df = pd.concat(bad_rows,axis =1).T
except:
    print("All players generated! Yay!")

# Generate ratings for each season

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np    
import os

seasons = list(range(1980,2026))
# seasons = list(range(1991,1992))
target_num_players = 250
weightings = {"PTS": 5,"AST": 3.5,"TRB": 3,"STL": 1.5,"BLK": 1.5,"eFG%": 2,"FT%": 1,"TOV": -1,"PF": -1}
stats = list(weightings.keys())


season_dir = 'season_ratings/'
if not os.path.exists(season_dir):
    os.makedirs(season_dir)


for i in range (len(seasons)):
    
    season_data = []
    ###############INSERT LOOOOOOOOOOOOOOP HERE########################
    season = seasons[i]
    season_string = f"{season-1}-{str(season%100).zfill(2)}"
    print(season_string)

        #######################GET ELIGIBLE PLAYERS########################
    players = data.query(f"From <= {season} and To >= {season}")

    for j in range(len(players)):
        try:
            row = players.iloc[j]
            player_data = pd.read_csv(f"player_dfs/{row.Letter}/{row.Extension}{row.Number}.csv", index_col = 'Season') 
            if season_string in player_data.index:
                season_row = player_data.loc[season_string]
                season_data.append(season_row)
        except:
            print('Couldnt load: ', row.Player, ' @ ', row.url)

    season_data = pd.concat(season_data,axis =1).T
    season_data = season_data.set_index("Player")
    season_data = season_data.fillna(0)
    if len(season_data) > target_num_players:
        min_games = int(season_data.sort_values('G', ascending = False).iloc[target_num_players].G)
        season_data = season_data.query(f"G >= {min_games}")

    scaler = StandardScaler()
    season_data[stats] = scaler.fit_transform(np.array(season_data[stats]))
    season_data = season_data.filter(['Player','Pos','Ht']+ stats)
    season_data[stats] = season_data[stats].multiply(list(weightings.values()))
    season_data['ChrissyMetric'] = season_data[stats].sum(axis = 1)
    season_data = season_data.sort_values("ChrissyMetric", ascending = False) 

    path = f'{season_dir}{season_string}.csv'
    season_data.to_csv(path)

In [1]:
import pandas as pd

season_data = pd.read_csv("season_ratings/2024-25.csv")
season_data.head(50)

,Player,Pos,Ht,PTS,AST,TRB,STL,BLK,eFG%,FT%,TOV,PF,ChrissyMetric
0,Nikola Jokic,C-F,6-11,12.841539,12.471418,8.566346,1.976444,1.363673,1.526857,0.349941,-1.855281,0.099657,37.340594
1,Anthony Davis,F-C,6-10,14.237572,0.418927,8.238823,2.849399,4.166011,0.765634,0.276842,-0.876095,1.175796,31.252909
2,Giannis Antetokounmpo,F-G,6-11,13.679159,6.445173,8.566346,-0.933406,-0.037496,1.590292,-0.704217,-1.561526,-1.514552,25.529773
3,Shai Gilgeous-Alexander,G,6-6,10.538084,6.789529,2.452580,2.849399,1.924141,-0.185894,0.780838,-0.386503,0.422498,25.184672
4,Trae Young,G,6-1,10.887093,15.742808,0.596615,1.394474,-0.597964,-0.518929,0.726976,-3.617816,0.207270,24.820527
5,James Harden,G,6-5,7.257406,14.193202,4.745242,2.558414,1.363673,-1.613187,0.576931,-4.009490,-0.438413,24.633779
6,Jayson Tatum,F,6-8,13.190548,3.518139,3.435149,3.140384,0.242738,0.733917,0.272995,-1.267770,-0.653641,22.612459
7,Paolo Banchero,F,6-10,12.283126,5.239923,4.745242,-0.642421,0.803205,0.305729,-0.269473,-0.778177,-0.546027,21.141128
8,Domantas Sabonis,C-F,6-10,7.048001,6.445173,9.330567,-0.351436,-0.597964,2.050198,0.392261,-1.561526,-2.267849,20.487425
9,Chet Holmgren,F-C,7-1,4.395538,0.763284,5.836986,1.394474,7.809051,0.955940,0.107562,-1.365688,-0.115571,19.781575


# DEBUG 